# Comparing Proportional Selection and SUS Selection
In this notebook, I will compare the results of using proportional selection, SUS selection, and tournament selection on the 2D step problem.

In [1]:
import matplotlib.pyplot as plt
from tqdm import tqdm

from leap_ec.algorithm import generational_ea
from leap_ec.individual import Individual
from leap_ec.representation import Representation
import leap_ec.ops as ops
from leap_ec import probe
from leap_ec.global_vars import context

from leap_ec.real_rep.problems import StepProblem
from leap_ec.real_rep.ops import mutate_gaussian
from leap_ec.real_rep.initializers import create_real_vector

In [2]:
MAX_GEN = 1000
POP_SIZE = 10

In [3]:
prop_file = open('./proportional_stats.csv', 'w')
prop_stats_probe = probe.FitnessStatsCSVProbe(stream=prop_file, context=context)
for i in tqdm(range(100)):
    prop_stats_probe.bsf_ind = None
    ea = generational_ea(max_generations=MAX_GEN, pop_size=POP_SIZE,
                         problem=StepProblem(maximize=True),
                         representation=Representation(
                             individual_cls=Individual,
                             initialize=create_real_vector(bounds=[[-5.12, 5.12]]*2)
                         ),
                         pipeline=[
                             ops.proportional_selection(offset='pop-min'),
                             ops.clone,
                             ops.uniform_crossover,
                             mutate_gaussian(std=0.5, expected_num_mutations='isotropic'),
                             ops.evaluate,
                             ops.pool(size=POP_SIZE),
                             prop_stats_probe
                         ])
    list(ea);
prop_file.close()

 53%|█████▎    | 53/100 [00:26<00:22,  2.06it/s]/home/thomas/Documents/ORNL/LEAP/leap_ec/ops.py:558: RuntimeWarning: invalid value encountered in true_divide
  proportions = values / population_total
100%|██████████| 100/100 [00:49<00:00,  2.04it/s]


In [4]:
sus_file = open('./SUS_stats.csv', 'w')
sus_stats_probe = probe.FitnessStatsCSVProbe(stream=sus_file, context=context)
for i in tqdm(range(100)):
    sus_stats_probe.bsf_ind = None
    ea = generational_ea(max_generations=MAX_GEN, pop_size=POP_SIZE,
                         problem=StepProblem(maximize=True),
                         representation=Representation(
                             individual_cls=Individual,
                             initialize=create_real_vector(bounds=[[-5.12, 5.12]]*2)
                         ),
                         pipeline=[
                             ops.sus_selection(offset='pop-min'),
                             ops.clone,
                             ops.uniform_crossover,
                             mutate_gaussian(std=0.5, expected_num_mutations='isotropic'),
                             ops.evaluate,
                             ops.pool(size=POP_SIZE),
                             sus_stats_probe
                         ])
    list(ea);
sus_file.close()

100%|██████████| 100/100 [00:42<00:00,  2.33it/s]


In [5]:
import pandas as pd

prop_df = pd.read_csv('proportional_stats.csv')
sus_df = pd.read_csv('SUS_stats.csv')

prop_df = prop_df.groupby('step').mean()
prop_df.to_csv('prop_avg_stats.csv')
print(prop_df.head())

       bsf   mean_fitness   std_fitness   min_fitness   max_fitness
step                                                               
0     6.09          1.549      3.069681         -3.67          6.09
1     6.80          3.333      2.278884         -0.73          6.59
2     7.45          4.607      1.778320          1.43          7.26
3     8.07          5.596      1.494322          3.08          7.90
4     8.71          6.538      1.322581          4.31          8.57
